# Transactions

- Combine transactions.csv and transaction_v2.csv.
- Add year & month column
- Then transform into parquet.

In [2]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType


In [39]:
from pyspark.sql.functions import countDistinct

In [12]:
from pyspark.sql.functions import lit

In [22]:
from pyspark.sql.functions import substring

In [3]:
os.getcwd()

'/app'

In [4]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .config("spark.driver.memory", "8g") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/17 07:53:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Read CSV file
v1 = spark.read.csv("data/transactions.csv/transactions.csv", header=True, inferSchema=True)

In [8]:
v2 = spark.read.csv("data/transactions_v2.csv/transactions_v2.csv", header=True, inferSchema=True)

In [10]:
v1.show(5)

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|YyO+tlZtAXYXoZhNr...|               41|               30|            129|               129|            1|        20150930|              20151101|        0|
|AZtu6Wl0gPojrEQYB...|               41|               30|            149|               149|            1|        20150930|              20151031|        0|
|UkDFI97Qb6+s2LWci...|               41|               30|            129|               129|            1|        20150930|              20160427|        0|
|M1C56ijxozNaGD0t2...|               39|            

In [11]:
v2.show(5)

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|++6eU4LsQ3UQ20ILS...|               32|               90|            298|               298|            0|        20170131|              20170504|        0|
|++lvGPJOinuin/8es...|               41|               30|            149|               149|            1|        20150809|              20190412|        0|
|+/GXNtXWQVfKrEDqY...|               36|               30|            180|               180|            1|        20170303|              20170422|        0|
|+/w1UrZwyka4C9oNH...|               36|            

In [13]:
# Add source file tracking
v1_tagged = v1.withColumn("source_file", lit("transactions.csv"))
v2_tagged = v2.withColumn("source_file", lit("transactions_v2.csv"))

In [14]:
# Union both datasets
combined = v1_tagged.union(v2_tagged)

In [15]:
# Deduplicate based on all original columns (excluding source_file)
data_columns = v1.columns  # Original columns without source_file

In [16]:
merged = combined.dropDuplicates(subset=data_columns)

In [18]:
merged.show(10)

[Stage 11:>                                                         (0 + 1) / 1]

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+----------------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|     source_file|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+----------------+
|dS2WKDR7me3ESNLko...|                1|                0|              0|                 0|            0|        20151202|              20100325|        0|transactions.csv|
|4tGbzhQiAimvXQe8g...|                3|                0|              0|               894|            0|        20150509|              20151127|        0|transactions.csv|
|LKDBe2Gvzg0GNhXti...|                8|                0|              0|               894|            0|        20150512| 

In [19]:
merged.count()

22975416

In [21]:
merged.select("transaction_date").distinct().orderBy("transaction_date").show()

[Stage 18:=============================================>          (17 + 4) / 21]

+----------------+
|transaction_date|
+----------------+
|        20150101|
|        20150102|
|        20150103|
|        20150104|
|        20150105|
|        20150106|
|        20150107|
|        20150108|
|        20150109|
|        20150110|
|        20150111|
|        20150112|
|        20150113|
|        20150114|
|        20150115|
|        20150116|
|        20150117|
|        20150118|
|        20150119|
|        20150120|
+----------------+
only showing top 20 rows



In [23]:
# Extract year and month
df_with_ym = merged.withColumn("year", substring("transaction_date", 1, 4)) \
                   .withColumn("month", substring("transaction_date", 5, 2))

# Count distinct years and months
df_with_ym.select("year").distinct().count(), df_with_ym.select("month").distinct().count()

(3, 12)

In [24]:
df_with_ym.show(10)

[Stage 35:>                                                         (0 + 1) / 1]

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+----------------+----+-----+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|     source_file|year|month|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+----------------+----+-----+
|dS2WKDR7me3ESNLko...|                1|                0|              0|                 0|            0|        20151202|              20100325|        0|transactions.csv|2015|   12|
|4tGbzhQiAimvXQe8g...|                3|                0|              0|               894|            0|        20150509|              20151127|        0|transactions.csv|2015|   05|
|LKDBe2Gvzg0GNhXti...|                8|                0|            

In [25]:
df_with_ym.write \
    .mode("overwrite") \
    .option("header", "true") \
    .partitionBy("year", "month") \
    .csv("datamart/bronze/transactions")

In [40]:
result = df_with_ym.agg(countDistinct("msno").alias("distinct_msno_count"))
result.show()

[Stage 69:==============================================>          (9 + 2) / 11]

+-------------------+
|distinct_msno_count|
+-------------------+
|            2426143|
+-------------------+



# Members

- Add year & month column
- Then transform into parquet.

In [26]:
# Read CSV file
df = spark.read.csv("data/members_v3.csv/members_v3.csv", header=True, inferSchema=True)

In [27]:
df.show(10)

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  NULL|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  NULL|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  NULL|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  NULL|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
|yLkV2gbZ4GLFwqTOX...|   4| 30|  male|             9|              20110916|
|jNCGK78YkTyId3H3w...|   1|  0|  NULL|             7|              20110916|
|WH5Jq4mgtfUFXh2yz...|   5| 34|  male|             9|              20110916|
|tKmbR4X5VXjHmxERr...|   5| 19|  male|             9|              20110917|
|I0yFvqMoNkM8ZNHb6...|  13| 63|  male|             9|              20110918|

In [30]:
df.count()

6769473

In [29]:
df.select("registration_init_time").distinct().orderBy("registration_init_time").show()

[Stage 42:=======>                                                  (1 + 7) / 8]

+----------------------+
|registration_init_time|
+----------------------+
|              20040326|
|              20040327|
|              20040328|
|              20040329|
|              20040330|
|              20040331|
|              20040401|
|              20040402|
|              20040403|
|              20040404|
|              20040405|
|              20040406|
|              20040407|
|              20040408|
|              20040409|
|              20040410|
|              20040411|
|              20040412|
|              20040413|
|              20040414|
+----------------------+
only showing top 20 rows



In [32]:
# Extract year and month
dfm_with_ym = df.withColumn("year", substring("registration_init_time", 1, 4)) \
                   .withColumn("month", substring("registration_init_time", 5, 2))

# Count distinct years and months
dfm_with_ym.select("year").distinct().count(), dfm_with_ym.select("month").distinct().count()

(14, 12)

In [33]:
dfm_with_ym.show(10)

+--------------------+----+---+------+--------------+----------------------+----+-----+
|                msno|city| bd|gender|registered_via|registration_init_time|year|month|
+--------------------+----+---+------+--------------+----------------------+----+-----+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  NULL|            11|              20110911|2011|   09|
|+tJonkh+O1CA796Fm...|   1|  0|  NULL|             7|              20110914|2011|   09|
|cV358ssn7a0f7jZOw...|   1|  0|  NULL|            11|              20110915|2011|   09|
|9bzDeJP6sQodK73K5...|   1|  0|  NULL|            11|              20110915|2011|   09|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|2011|   09|
|yLkV2gbZ4GLFwqTOX...|   4| 30|  male|             9|              20110916|2011|   09|
|jNCGK78YkTyId3H3w...|   1|  0|  NULL|             7|              20110916|2011|   09|
|WH5Jq4mgtfUFXh2yz...|   5| 34|  male|             9|              20110916|2011|   09|
|tKmbR4X5VXjHmxERr...|   5| 19| 

In [36]:
result = dfm_with_ym.groupBy("year", "month") \
           .count() \
           .orderBy("year", "month")
result.show()

[Stage 61:>                                                         (0 + 8) / 8]

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2004|   03| 4258|
|2004|   04| 1417|
|2004|   05|  436|
|2004|   06|  550|
|2004|   07| 4713|
|2004|   08| 2474|
|2004|   09| 3323|
|2004|   10| 4547|
|2004|   11| 3075|
|2004|   12| 1441|
|2005|   01| 1382|
|2005|   02| 2520|
|2005|   03| 2752|
|2005|   04| 2132|
|2005|   05| 1939|
|2005|   06| 1993|
|2005|   07| 2177|
|2005|   08| 2053|
|2005|   09| 5054|
|2005|   10| 8642|
+----+-----+-----+
only showing top 20 rows



In [37]:
result = dfm_with_ym.groupBy("year") \
           .count() \
           .orderBy("year")
result.show()

[Stage 64:>                                                         (0 + 8) / 8]

+----+-------+
|year|  count|
+----+-------+
|2004|  26234|
|2005|  41349|
|2006|  53953|
|2007|  89830|
|2008|  67690|
|2009|  63633|
|2010| 115075|
|2011| 179051|
|2012| 283190|
|2013| 524722|
|2014| 975776|
|2015|1620525|
|2016|2246761|
|2017| 481684|
+----+-------+



No need to split to folders